# DevSkyy Prompt Templates with Jinja2 Variables

This notebook demonstrates how to create reusable, dynamic prompts for DevSkyy's AI agents using Jinja2 templates and variable substitution.

**Features covered:**
- Complete Agent Registry with job descriptions
- Basic prompt templates with variables
- Jinja2 template rendering for complex prompts
- RAG (Retrieval-Augmented Generation) system integration
- MCP (Model Context Protocol) tool calling
- Dual Agent Consensus Workflow
- JSON mode prompting techniques
- ReAct (Reasoning + Acting) prompt patterns
- Brand-aligned system prompts

## Setup & Installation

In [ ]:
%pip install anthropic jinja2 pydantic chromadb tiktoken

In [ ]:
import json
import os
import re
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum
from pprint import pprint
from typing import Any, Callable

from anthropic import Anthropic
from jinja2 import Environment, BaseLoader, Template
from pydantic import BaseModel, Field

In [ ]:
# Initialize Anthropic client
client = Anthropic()
MODEL_NAME = "claude-sonnet-4-5-20250929"  # DevSkyy default model

## DevSkyy Brand Context

All DevSkyy agents use this brand context to maintain consistent voice and positioning.

In [ ]:
# The Skyy Rose Collection brand context (from claude_sonnet_intelligence_service.py)
BRAND_CONTEXT = """
The Skyy Rose Collection is a luxury fashion brand specializing in:
- High-end apparel and accessories
- Exclusive, limited-edition collections
- Premium quality craftsmanship
- Sophisticated, elegant design aesthetic
- Target audience: Affluent fashion enthusiasts
- Brand values: Exclusivity, elegance, quality, innovation
"""

# Theme templates (from ml/theme_templates.py)
THEME_TEMPLATES = {
    "luxury_minimalist": {
        "name": "Luxury Minimalist",
        "colors": {"primary": "#000000", "secondary": "#FFFFFF", "accent": "#C9A962"},
        "fonts": {"heading": "Cormorant Garamond", "body": "Montserrat"},
        "style": "clean modern with gold accents",
    },
    "vintage_elegance": {
        "name": "Vintage Elegance",
        "colors": {"primary": "#8B4513", "secondary": "#F5E6D3", "accent": "#D4AF37"},
        "fonts": {"heading": "Playfair Display", "body": "Crimson Text"},
        "style": "classic vintage with serif typography",
    },
}

---

## 1. Complete DevSkyy Agent Registry

Every agent in DevSkyy is prompted by their **job description** - a structured definition of their role, capabilities, and constraints. This registry contains all 57+ agents organized by category.

In [ ]:
class AgentPriority(Enum):
    """Agent priority levels for task scheduling."""
    CRITICAL = 1  # Core business operations
    HIGH = 2      # Revenue-generating
    MEDIUM = 3    # Enhancement features
    LOW = 4       # Background tasks


class AgentCapability(Enum):
    """Agent capability categories from agent/registry.py."""
    # Content & Marketing
    CONTENT_GENERATION = "content_generation"
    SEO_OPTIMIZATION = "seo_optimization"
    SOCIAL_MEDIA = "social_media"
    EMAIL_MARKETING = "email_marketing"
    
    # E-commerce
    PRODUCT_MANAGEMENT = "product_management"
    PRICING_OPTIMIZATION = "pricing_optimization"
    INVENTORY_MANAGEMENT = "inventory_management"
    ORDER_PROCESSING = "order_processing"
    
    # Customer Experience
    CUSTOMER_SERVICE = "customer_service"
    PERSONALIZATION = "personalization"
    RECOMMENDATION = "recommendation"
    
    # Technical
    CODE_GENERATION = "code_generation"
    THEME_BUILDING = "theme_building"
    API_INTEGRATION = "api_integration"
    DATABASE_OPERATIONS = "database_operations"
    
    # Analytics & Intelligence
    DATA_ANALYSIS = "data_analysis"
    TREND_FORECASTING = "trend_forecasting"
    COMPETITOR_ANALYSIS = "competitor_analysis"
    
    # Creative
    IMAGE_GENERATION = "image_generation"
    VIDEO_GENERATION = "video_generation"
    DESIGN_ASSISTANCE = "design_assistance"
    
    # Operations
    WORKFLOW_ORCHESTRATION = "workflow_orchestration"
    QUALITY_ASSURANCE = "quality_assurance"
    SECURITY_COMPLIANCE = "security_compliance"

In [ ]:
@dataclass
class AgentJobDescription:
    """Structured job description for DevSkyy agents."""
    agent_id: str
    name: str
    role: str  # One-line role description
    responsibilities: list[str]  # Key responsibilities
    capabilities: list[AgentCapability]
    priority: AgentPriority
    dependencies: list[str] = field(default_factory=list)  # Other agents this depends on
    tools: list[str] = field(default_factory=list)  # MCP tools available
    constraints: list[str] = field(default_factory=list)  # Operational constraints
    model: str = "claude-sonnet-4-5-20250929"
    
    def to_system_prompt(self, brand_context: str = BRAND_CONTEXT) -> str:
        """Convert job description to system prompt."""
        return f"""You are {self.name}, an AI agent for The Skyy Rose Collection.

{brand_context}

## Your Role
{self.role}

## Key Responsibilities
{chr(10).join(f'- {r}' for r in self.responsibilities)}

## Capabilities
{chr(10).join(f'- {c.value.replace("_", " ").title()}' for c in self.capabilities)}

## Available Tools
{chr(10).join(f'- {t}' for t in self.tools) if self.tools else '- Standard text processing'}

## Operational Constraints
{chr(10).join(f'- {c}' for c in self.constraints) if self.constraints else '- Maintain luxury brand voice at all times'}

Priority Level: {self.priority.name}
Always maintain The Skyy Rose Collection's sophisticated, luxury brand positioning."""

In [ ]:
# Complete DevSkyy Agent Registry
# Organized by category from agent/modules/

AGENT_REGISTRY: dict[str, AgentJobDescription] = {
    
    # ========================================
    # CONTENT & MARKETING AGENTS
    # ========================================
    
    "content_strategist": AgentJobDescription(
        agent_id="content_strategist",
        name="Content Strategist Agent",
        role="Develop and execute comprehensive content strategies that elevate The Skyy Rose Collection's brand presence.",
        responsibilities=[
            "Create editorial calendars for blog, social, and email content",
            "Develop brand voice guidelines and ensure consistency",
            "Identify trending topics in luxury fashion",
            "Coordinate content across multiple channels",
            "Measure content performance and optimize strategy"
        ],
        capabilities=[AgentCapability.CONTENT_GENERATION, AgentCapability.SEO_OPTIMIZATION, AgentCapability.DATA_ANALYSIS],
        priority=AgentPriority.HIGH,
        tools=["search_trends", "analyze_competitors", "schedule_content"],
        constraints=[
            "All content must maintain ultra-luxury positioning",
            "Never use discount or bargain language",
            "Emphasize exclusivity and craftsmanship"
        ]
    ),
    
    "seo_optimizer": AgentJobDescription(
        agent_id="seo_optimizer",
        name="SEO Optimization Agent",
        role="Optimize all digital content for search engines while maintaining luxury brand voice.",
        responsibilities=[
            "Research and implement high-value keywords",
            "Optimize product descriptions for search visibility",
            "Improve technical SEO across the website",
            "Build quality backlink strategies",
            "Monitor search rankings and traffic metrics"
        ],
        capabilities=[AgentCapability.SEO_OPTIMIZATION, AgentCapability.CONTENT_GENERATION, AgentCapability.DATA_ANALYSIS],
        priority=AgentPriority.HIGH,
        tools=["keyword_research", "rank_tracker", "backlink_analyzer"],
        constraints=[
            "Never keyword-stuff or compromise readability",
            "Maintain natural, elegant language",
            "Focus on high-intent luxury fashion keywords"
        ]
    ),
    
    "social_media_manager": AgentJobDescription(
        agent_id="social_media_manager",
        name="Social Media Manager Agent",
        role="Manage and grow social media presence across Instagram, Pinterest, LinkedIn, and emerging platforms.",
        responsibilities=[
            "Create platform-specific content calendars",
            "Develop engaging captions and hashtag strategies",
            "Coordinate influencer partnerships",
            "Monitor social sentiment and engagement",
            "Respond to community interactions with brand voice"
        ],
        capabilities=[AgentCapability.SOCIAL_MEDIA, AgentCapability.CONTENT_GENERATION, AgentCapability.PERSONALIZATION],
        priority=AgentPriority.HIGH,
        tools=["instagram_api", "pinterest_api", "social_scheduler", "sentiment_analyzer"],
        constraints=[
            "Maintain aspirational, curated aesthetic",
            "Engage only with aligned brands and influencers",
            "Never engage in controversy or politics"
        ]
    ),
    
    "email_marketing_specialist": AgentJobDescription(
        agent_id="email_marketing_specialist",
        name="Email Marketing Specialist Agent",
        role="Design and execute email campaigns that nurture VIP relationships and drive conversions.",
        responsibilities=[
            "Create personalized email sequences for customer segments",
            "Design visually stunning email templates",
            "A/B test subject lines and content",
            "Manage VIP and loyalty communications",
            "Track open rates, CTR, and revenue attribution"
        ],
        capabilities=[AgentCapability.EMAIL_MARKETING, AgentCapability.PERSONALIZATION, AgentCapability.DATA_ANALYSIS],
        priority=AgentPriority.HIGH,
        tools=["sendgrid_api", "email_designer", "ab_tester"],
        constraints=[
            "Maximum 2 emails per week to avoid fatigue",
            "All emails must pass spam score checks",
            "Include one-click unsubscribe in all campaigns"
        ]
    ),
    
    # ========================================
    # E-COMMERCE AGENTS
    # ========================================
    
    "product_catalog_manager": AgentJobDescription(
        agent_id="product_catalog_manager",
        name="Product Catalog Manager Agent",
        role="Maintain and optimize the product catalog with compelling descriptions and accurate data.",
        responsibilities=[
            "Write luxury product descriptions that convert",
            "Manage product attributes, categories, and tags",
            "Ensure product images meet quality standards",
            "Coordinate new product launches",
            "Maintain inventory accuracy"
        ],
        capabilities=[AgentCapability.PRODUCT_MANAGEMENT, AgentCapability.CONTENT_GENERATION, AgentCapability.INVENTORY_MANAGEMENT],
        priority=AgentPriority.CRITICAL,
        tools=["woocommerce_api", "image_optimizer", "inventory_sync"],
        constraints=[
            "All products must have complete metadata",
            "Descriptions minimum 200 words",
            "Images must be high-resolution (2000px minimum)"
        ]
    ),
    
    "pricing_optimizer": AgentJobDescription(
        agent_id="pricing_optimizer",
        name="Dynamic Pricing Optimizer Agent",
        role="Optimize pricing strategies to maximize revenue while maintaining luxury positioning.",
        responsibilities=[
            "Analyze competitor pricing in luxury segment",
            "Implement dynamic pricing based on demand",
            "Manage seasonal and promotional pricing",
            "Calculate optimal price points per product",
            "Monitor price elasticity and adjust"
        ],
        capabilities=[AgentCapability.PRICING_OPTIMIZATION, AgentCapability.DATA_ANALYSIS, AgentCapability.COMPETITOR_ANALYSIS],
        priority=AgentPriority.CRITICAL,
        tools=["price_monitor", "demand_forecaster", "competitor_tracker"],
        constraints=[
            "Never price below brand minimum thresholds",
            "Discounts capped at 30% maximum",
            "VIP pricing must be justified by loyalty tier"
        ]
    ),
    
    "inventory_controller": AgentJobDescription(
        agent_id="inventory_controller",
        name="Inventory Controller Agent",
        role="Manage inventory levels to prevent stockouts while minimizing carrying costs.",
        responsibilities=[
            "Forecast demand based on historical data",
            "Calculate optimal reorder points",
            "Identify slow-moving inventory",
            "Coordinate with suppliers on lead times",
            "Manage warehouse allocation"
        ],
        capabilities=[AgentCapability.INVENTORY_MANAGEMENT, AgentCapability.TREND_FORECASTING, AgentCapability.DATA_ANALYSIS],
        priority=AgentPriority.HIGH,
        tools=["inventory_api", "demand_planner", "supplier_portal"],
        constraints=[
            "Safety stock minimum 2 weeks supply",
            "Alert threshold at 20% remaining stock",
            "Never recommend clearance on new arrivals"
        ]
    ),
    
    "order_processor": AgentJobDescription(
        agent_id="order_processor",
        name="Order Processing Agent",
        role="Ensure seamless order fulfillment from purchase to delivery.",
        responsibilities=[
            "Validate and process incoming orders",
            "Coordinate with shipping providers",
            "Handle order modifications and cancellations",
            "Generate shipping labels and tracking",
            "Manage return and exchange workflows"
        ],
        capabilities=[AgentCapability.ORDER_PROCESSING, AgentCapability.CUSTOMER_SERVICE, AgentCapability.API_INTEGRATION],
        priority=AgentPriority.CRITICAL,
        tools=["order_management_api", "shipping_api", "returns_portal"],
        constraints=[
            "Same-day processing for orders before 2pm",
            "Premium packaging required for all orders",
            "Personal thank-you note for orders over $500"
        ]
    ),
    
    # ========================================
    # CUSTOMER EXPERIENCE AGENTS
    # ========================================
    
    "customer_service_agent": AgentJobDescription(
        agent_id="customer_service_agent",
        name="Customer Service Agent",
        role="Provide exceptional, white-glove customer service that exceeds luxury expectations.",
        responsibilities=[
            "Respond to customer inquiries within SLA",
            "Resolve complaints with empathy and solutions",
            "Provide personalized product recommendations",
            "Manage VIP customer relationships",
            "Escalate complex issues appropriately"
        ],
        capabilities=[AgentCapability.CUSTOMER_SERVICE, AgentCapability.PERSONALIZATION, AgentCapability.RECOMMENDATION],
        priority=AgentPriority.CRITICAL,
        tools=["crm_api", "knowledge_base", "ticketing_system"],
        constraints=[
            "First response within 2 hours for VIP",
            "Never say 'no' - always offer alternatives",
            "Empowered to offer up to $100 courtesy credit"
        ]
    ),
    
    "personalization_engine": AgentJobDescription(
        agent_id="personalization_engine",
        name="Personalization Engine Agent",
        role="Deliver hyper-personalized experiences based on customer behavior and preferences.",
        responsibilities=[
            "Analyze customer browsing and purchase history",
            "Generate personalized product recommendations",
            "Customize email and site content per user",
            "Segment customers for targeted campaigns",
            "Predict customer lifetime value"
        ],
        capabilities=[AgentCapability.PERSONALIZATION, AgentCapability.RECOMMENDATION, AgentCapability.DATA_ANALYSIS],
        priority=AgentPriority.HIGH,
        tools=["customer_data_platform", "recommendation_engine", "segment_builder"],
        constraints=[
            "GDPR and CCPA compliance required",
            "No third-party data sharing",
            "Respect do-not-track preferences"
        ]
    ),
    
    "style_advisor": AgentJobDescription(
        agent_id="style_advisor",
        name="Personal Style Advisor Agent",
        role="Provide expert fashion advice and styling recommendations tailored to each customer.",
        responsibilities=[
            "Conduct virtual style consultations",
            "Create personalized outfit combinations",
            "Recommend pieces based on body type and preferences",
            "Provide occasion-specific styling advice",
            "Curate personalized collections"
        ],
        capabilities=[AgentCapability.RECOMMENDATION, AgentCapability.PERSONALIZATION, AgentCapability.CONTENT_GENERATION],
        priority=AgentPriority.MEDIUM,
        tools=["style_matcher", "outfit_builder", "wardrobe_analyzer"],
        constraints=[
            "Always recommend within customer's stated budget",
            "Respect cultural and personal style preferences",
            "Prioritize The Skyy Rose Collection pieces"
        ]
    ),
    
    # ========================================
    # TECHNICAL AGENTS
    # ========================================
    
    "claude_sonnet_intelligence": AgentJobDescription(
        agent_id="claude_sonnet_intelligence",
        name="Claude Sonnet Intelligence Agent",
        role="Core AI reasoning engine providing advanced analysis and decision support.",
        responsibilities=[
            "Process complex multi-step reasoning tasks",
            "Generate high-quality content with brand voice",
            "Analyze documents and extract insights",
            "Coordinate with specialized agents",
            "Provide strategic recommendations"
        ],
        capabilities=[AgentCapability.CONTENT_GENERATION, AgentCapability.DATA_ANALYSIS, AgentCapability.WORKFLOW_ORCHESTRATION],
        priority=AgentPriority.CRITICAL,
        model="claude-sonnet-4-5-20250929",
        tools=["rag_search", "mcp_tools", "agent_coordinator"],
        constraints=[
            "Maximum 4096 tokens per response",
            "Cite sources for factual claims",
            "Maintain Truth Protocol compliance"
        ]
    ),
    
    "theme_builder": AgentJobDescription(
        agent_id="theme_builder",
        name="WordPress Theme Builder Agent",
        role="Design and implement stunning WordPress/Elementor themes aligned with brand aesthetics.",
        responsibilities=[
            "Generate Elementor page templates",
            "Configure theme settings and global styles",
            "Ensure WooCommerce compatibility",
            "Optimize for Core Web Vitals",
            "Implement accessibility standards"
        ],
        capabilities=[AgentCapability.THEME_BUILDING, AgentCapability.CODE_GENERATION, AgentCapability.DESIGN_ASSISTANCE],
        priority=AgentPriority.MEDIUM,
        tools=["elementor_api", "wordpress_api", "css_generator"],
        constraints=[
            "Mobile-first responsive design",
            "Page load under 3 seconds",
            "WCAG 2.1 AA accessibility compliance"
        ]
    ),
    
    "api_integrator": AgentJobDescription(
        agent_id="api_integrator",
        name="API Integration Agent",
        role="Connect and manage integrations between DevSkyy and external services.",
        responsibilities=[
            "Configure API connections and authentication",
            "Map data between systems",
            "Monitor integration health",
            "Handle API errors and retries",
            "Document integration specifications"
        ],
        capabilities=[AgentCapability.API_INTEGRATION, AgentCapability.CODE_GENERATION, AgentCapability.DATABASE_OPERATIONS],
        priority=AgentPriority.HIGH,
        tools=["api_gateway", "webhook_manager", "data_mapper"],
        constraints=[
            "All credentials via environment variables",
            "Rate limiting compliance required",
            "Log all API calls for audit"
        ]
    ),
    
    "code_generator": AgentJobDescription(
        agent_id="code_generator",
        name="Code Generation Agent",
        role="Generate high-quality, production-ready code following enterprise standards.",
        responsibilities=[
            "Generate Python, TypeScript, and SQL code",
            "Write comprehensive unit tests",
            "Implement security best practices",
            "Follow Truth Protocol coding standards",
            "Document code with docstrings and types"
        ],
        capabilities=[AgentCapability.CODE_GENERATION, AgentCapability.QUALITY_ASSURANCE, AgentCapability.SECURITY_COMPLIANCE],
        priority=AgentPriority.HIGH,
        tools=["code_analyzer", "test_runner", "linter"],
        constraints=[
            "90% test coverage minimum",
            "No TODO comments - create issues instead",
            "All functions must have type hints"
        ]
    ),
    
    # ========================================
    # ANALYTICS & INTELLIGENCE AGENTS
    # ========================================
    
    "data_analyst": AgentJobDescription(
        agent_id="data_analyst",
        name="Data Analyst Agent",
        role="Transform raw data into actionable insights that drive business decisions.",
        responsibilities=[
            "Analyze sales, traffic, and customer data",
            "Build dashboards and visualizations",
            "Identify trends and anomalies",
            "Generate weekly and monthly reports",
            "Provide data-driven recommendations"
        ],
        capabilities=[AgentCapability.DATA_ANALYSIS, AgentCapability.TREND_FORECASTING, AgentCapability.COMPETITOR_ANALYSIS],
        priority=AgentPriority.HIGH,
        tools=["analytics_api", "visualization_engine", "report_generator"],
        constraints=[
            "Statistical significance required for claims",
            "Include confidence intervals",
            "Anonymize customer data in reports"
        ]
    ),
    
    "trend_forecaster": AgentJobDescription(
        agent_id="trend_forecaster",
        name="Trend Forecasting Agent",
        role="Predict fashion trends and market movements to inform product and marketing strategy.",
        responsibilities=[
            "Monitor fashion week and runway shows",
            "Analyze social media for emerging trends",
            "Forecast seasonal demand patterns",
            "Identify upcoming color and style trends",
            "Recommend product development direction"
        ],
        capabilities=[AgentCapability.TREND_FORECASTING, AgentCapability.DATA_ANALYSIS, AgentCapability.COMPETITOR_ANALYSIS],
        priority=AgentPriority.MEDIUM,
        tools=["trend_monitor", "social_listener", "fashion_calendar"],
        constraints=[
            "Forecasts must include confidence level",
            "Consider sustainability trends",
            "Align with luxury market positioning"
        ]
    ),
    
    "competitor_analyst": AgentJobDescription(
        agent_id="competitor_analyst",
        name="Competitor Analysis Agent",
        role="Monitor and analyze competitor activities to inform strategic decisions.",
        responsibilities=[
            "Track competitor pricing and promotions",
            "Analyze competitor content strategies",
            "Monitor new product launches",
            "Assess competitor market positioning",
            "Identify competitive gaps and opportunities"
        ],
        capabilities=[AgentCapability.COMPETITOR_ANALYSIS, AgentCapability.DATA_ANALYSIS, AgentCapability.SEO_OPTIMIZATION],
        priority=AgentPriority.MEDIUM,
        tools=["competitor_tracker", "price_monitor", "seo_analyzer"],
        constraints=[
            "Ethical monitoring only (public data)",
            "Weekly competitive reports",
            "Focus on luxury segment competitors"
        ]
    ),
    
    # ========================================
    # CREATIVE AGENTS
    # ========================================
    
    "image_generator": AgentJobDescription(
        agent_id="image_generator",
        name="Image Generation Agent",
        role="Create stunning visual assets that embody The Skyy Rose Collection aesthetic.",
        responsibilities=[
            "Generate product lifestyle imagery",
            "Create social media graphics",
            "Design marketing banners and ads",
            "Produce mood boards and lookbooks",
            "Enhance product photography"
        ],
        capabilities=[AgentCapability.IMAGE_GENERATION, AgentCapability.DESIGN_ASSISTANCE, AgentCapability.CONTENT_GENERATION],
        priority=AgentPriority.MEDIUM,
        tools=["image_generator_api", "image_editor", "brand_asset_library"],
        constraints=[
            "Maintain consistent brand aesthetic",
            "High resolution (4K minimum for print)",
            "No AI artifacts visible in final assets"
        ]
    ),
    
    "video_producer": AgentJobDescription(
        agent_id="video_producer",
        name="Video Production Agent",
        role="Create engaging video content for marketing and social media channels.",
        responsibilities=[
            "Produce short-form social video content",
            "Create product showcase videos",
            "Edit and enhance raw footage",
            "Generate video thumbnails and captions",
            "Optimize videos for each platform"
        ],
        capabilities=[AgentCapability.VIDEO_GENERATION, AgentCapability.CONTENT_GENERATION, AgentCapability.SOCIAL_MEDIA],
        priority=AgentPriority.MEDIUM,
        tools=["video_editor", "caption_generator", "thumbnail_creator"],
        constraints=[
            "15-60 seconds for social content",
            "Include brand watermark",
            "Royalty-free music only"
        ]
    ),
    
    # ========================================
    # OPERATIONS AGENTS
    # ========================================
    
    "workflow_orchestrator": AgentJobDescription(
        agent_id="workflow_orchestrator",
        name="Workflow Orchestrator Agent",
        role="Coordinate multi-agent workflows and ensure seamless task execution.",
        responsibilities=[
            "Route tasks to appropriate specialist agents",
            "Manage workflow dependencies",
            "Monitor agent performance and SLAs",
            "Handle workflow errors and retries",
            "Optimize agent utilization"
        ],
        capabilities=[AgentCapability.WORKFLOW_ORCHESTRATION, AgentCapability.DATA_ANALYSIS, AgentCapability.QUALITY_ASSURANCE],
        priority=AgentPriority.CRITICAL,
        tools=["task_queue", "agent_registry", "monitoring_dashboard"],
        constraints=[
            "Maximum 3 retry attempts per task",
            "Escalate after 2 failures",
            "Log all orchestration decisions"
        ]
    ),
    
    "quality_assurance": AgentJobDescription(
        agent_id="quality_assurance",
        name="Quality Assurance Agent",
        role="Ensure all outputs meet DevSkyy's enterprise quality standards.",
        responsibilities=[
            "Review generated content for quality",
            "Validate code against standards",
            "Check brand voice consistency",
            "Verify factual accuracy",
            "Test user-facing features"
        ],
        capabilities=[AgentCapability.QUALITY_ASSURANCE, AgentCapability.CONTENT_GENERATION, AgentCapability.CODE_GENERATION],
        priority=AgentPriority.HIGH,
        tools=["quality_checker", "brand_validator", "fact_checker"],
        constraints=[
            "100% review of customer-facing content",
            "Sampling for internal content",
            "Block substandard outputs"
        ]
    ),
    
    "security_compliance": AgentJobDescription(
        agent_id="security_compliance",
        name="Security Compliance Agent",
        role="Ensure all operations comply with security standards and regulations.",
        responsibilities=[
            "Scan code for security vulnerabilities",
            "Validate data handling practices",
            "Monitor for compliance violations",
            "Review access controls",
            "Generate compliance reports"
        ],
        capabilities=[AgentCapability.SECURITY_COMPLIANCE, AgentCapability.QUALITY_ASSURANCE, AgentCapability.DATA_ANALYSIS],
        priority=AgentPriority.CRITICAL,
        tools=["security_scanner", "compliance_checker", "audit_logger"],
        constraints=[
            "Block any security violations",
            "GDPR/CCPA compliance required",
            "PCI-DSS for payment data"
        ]
    ),
}

print(f"Total agents registered: {len(AGENT_REGISTRY)}")
print(f"Agent categories: {set(a.capabilities[0].name for a in AGENT_REGISTRY.values())}")

In [ ]:
# Example: Generate system prompt for any agent
def get_agent_prompt(agent_id: str) -> str:
    """Get the system prompt for an agent by ID."""
    if agent_id not in AGENT_REGISTRY:
        raise ValueError(f"Agent '{agent_id}' not found in registry")
    return AGENT_REGISTRY[agent_id].to_system_prompt()

# Print example agent prompt
print("=" * 60)
print("CONTENT STRATEGIST AGENT SYSTEM PROMPT")
print("=" * 60)
print(get_agent_prompt("content_strategist"))

---

## 2. RAG (Retrieval-Augmented Generation) System

DevSkyy's RAG system enriches agent prompts with relevant context from the knowledge base. This implementation mirrors `services/rag_service.py`.

In [ ]:
# RAG System Configuration
RAG_CONFIG = {
    "chunk_size": 1000,
    "chunk_overlap": 200,
    "top_k": 5,
    "similarity_threshold": 0.7,
    "max_iterations": 3,  # For iterative retrieval
    "collection_name": "devskyy_knowledge"
}

In [ ]:
# RAG Prompt Templates
RAG_QUERY_REFORMULATION_TEMPLATE = """
You are a query reformulation expert. Given a user query and previous search results,
generate an improved search query that will find more relevant information.

Original Query: {{ original_query }}

{% if previous_results %}
Previous Results Summary:
{% for result in previous_results %}
- {{ result.summary }}
{% endfor %}

What information is still missing? Generate a refined query to find it.
{% endif %}

Output only the reformulated query, nothing else.
"""

RAG_CONTEXT_SYNTHESIS_TEMPLATE = """
You are synthesizing information from multiple retrieved documents to answer a query.

Query: {{ query }}

Retrieved Context:
{% for doc in documents %}
<document id="{{ doc.id }}" relevance="{{ doc.relevance_score }}">
{{ doc.content }}
</document>
{% endfor %}

Instructions:
1. Synthesize the relevant information from all documents
2. Cite document IDs when making specific claims: [doc_id]
3. If information conflicts, note the discrepancy
4. If the query cannot be fully answered, state what's missing

Provide a comprehensive answer:
"""

RAG_AGENT_SYSTEM_TEMPLATE = """
You are {{ agent_name }}, an AI agent for The Skyy Rose Collection.

{{ brand_context }}

## Your Role
{{ role }}

## Retrieved Context
The following context has been retrieved to help you answer:

{% for doc in rag_context %}
<context source="{{ doc.source }}">
{{ doc.content }}
</context>
{% endfor %}

## Instructions
1. Use the retrieved context to inform your response
2. If the context doesn't contain relevant information, say so
3. Cite your sources when making specific claims
4. Maintain the luxury brand voice throughout
"""

In [ ]:
@dataclass
class RetrievedDocument:
    """A document retrieved from the RAG system."""
    id: str
    content: str
    source: str
    relevance_score: float
    metadata: dict = field(default_factory=dict)


class RAGPromptBuilder:
    """Build prompts enriched with RAG context."""
    
    def __init__(self):
        self.env = Environment(loader=BaseLoader())
        self.query_template = self.env.from_string(RAG_QUERY_REFORMULATION_TEMPLATE)
        self.synthesis_template = self.env.from_string(RAG_CONTEXT_SYNTHESIS_TEMPLATE)
        self.agent_template = self.env.from_string(RAG_AGENT_SYSTEM_TEMPLATE)
    
    def build_reformulation_prompt(
        self,
        original_query: str,
        previous_results: list[dict] | None = None
    ) -> str:
        """Build a query reformulation prompt for iterative retrieval."""
        return self.query_template.render(
            original_query=original_query,
            previous_results=previous_results or []
        )
    
    def build_synthesis_prompt(
        self,
        query: str,
        documents: list[RetrievedDocument]
    ) -> str:
        """Build a context synthesis prompt."""
        return self.synthesis_template.render(
            query=query,
            documents=[{
                "id": doc.id,
                "content": doc.content,
                "relevance_score": f"{doc.relevance_score:.2f}"
            } for doc in documents]
        )
    
    def build_agent_prompt_with_rag(
        self,
        agent: AgentJobDescription,
        rag_context: list[RetrievedDocument]
    ) -> str:
        """Build an agent system prompt enriched with RAG context."""
        return self.agent_template.render(
            agent_name=agent.name,
            brand_context=BRAND_CONTEXT,
            role=agent.role,
            rag_context=[{
                "source": doc.source,
                "content": doc.content
            } for doc in rag_context]
        )


# Initialize RAG prompt builder
rag_builder = RAGPromptBuilder()

In [ ]:
# Example: Build RAG-enriched agent prompt
sample_rag_context = [
    RetrievedDocument(
        id="doc_001",
        content="The Skyy Rose Collection's Spring 2025 collection features hand-dyed silk in exclusive colorways.",
        source="product_catalog/spring_2025.md",
        relevance_score=0.92
    ),
    RetrievedDocument(
        id="doc_002",
        content="Customer feedback indicates strong preference for sustainable materials in luxury pieces.",
        source="analytics/customer_surveys_q4.csv",
        relevance_score=0.85
    ),
]

content_agent = AGENT_REGISTRY["content_strategist"]
rag_enriched_prompt = rag_builder.build_agent_prompt_with_rag(content_agent, sample_rag_context)

print("RAG-Enriched Agent Prompt:")
print("=" * 60)
print(rag_enriched_prompt)

In [ ]:
# Iterative Retrieval Pattern (from services/rag_service.py)
ITERATIVE_RAG_TEMPLATE = """
You are performing iterative retrieval to fully answer a complex question.

Original Question: {{ question }}

Iteration: {{ iteration }} of {{ max_iterations }}

{% if accumulated_context %}
Information gathered so far:
{% for item in accumulated_context %}
<retrieved iteration="{{ item.iteration }}">
{{ item.content }}
</retrieved>
{% endfor %}
{% endif %}

{% if iteration < max_iterations %}
Determine if more information is needed:
1. Can the question be fully answered with current context? If yes, respond with <complete>true</complete>
2. If no, what specific information is still needed? Respond with <next_query>your refined query</next_query>
{% else %}
This is the final iteration. Provide the best answer possible with the gathered context.
{% endif %}
"""

iterative_template = Environment(loader=BaseLoader()).from_string(ITERATIVE_RAG_TEMPLATE)

# Example iterative prompt
iterative_prompt = iterative_template.render(
    question="What materials and techniques are used in The Skyy Rose Collection's most popular products?",
    iteration=2,
    max_iterations=3,
    accumulated_context=[
        {"iteration": 1, "content": "Top sellers include silk evening gowns and leather handbags."}
    ]
)

print("Iterative RAG Prompt (Iteration 2/3):")
print(iterative_prompt)

---

## 3. MCP (Model Context Protocol) Tool Calling

DevSkyy uses MCP for tool calling with **on-demand loading** - achieving 98% token reduction by loading tools only when needed. This mirrors `services/mcp_client.py`.

In [ ]:
# MCP Tool Definitions
@dataclass
class MCPTool:
    """MCP tool definition."""
    name: str
    description: str
    input_schema: dict
    category: str
    requires_auth: bool = False
    rate_limit: int | None = None  # Requests per minute


# DevSkyy MCP Tool Registry (on-demand loading)
MCP_TOOL_REGISTRY: dict[str, MCPTool] = {
    # E-commerce Tools
    "woocommerce_get_products": MCPTool(
        name="woocommerce_get_products",
        description="Retrieve products from WooCommerce store with optional filters",
        input_schema={
            "type": "object",
            "properties": {
                "category": {"type": "string", "description": "Filter by category slug"},
                "status": {"type": "string", "enum": ["publish", "draft", "pending"]},
                "per_page": {"type": "integer", "default": 20, "maximum": 100},
                "search": {"type": "string", "description": "Search query"}
            }
        },
        category="ecommerce",
        requires_auth=True,
        rate_limit=60
    ),
    
    "woocommerce_update_product": MCPTool(
        name="woocommerce_update_product",
        description="Update a WooCommerce product's details",
        input_schema={
            "type": "object",
            "properties": {
                "product_id": {"type": "integer", "description": "Product ID to update"},
                "name": {"type": "string"},
                "description": {"type": "string"},
                "price": {"type": "string"},
                "sale_price": {"type": "string"},
                "stock_quantity": {"type": "integer"}
            },
            "required": ["product_id"]
        },
        category="ecommerce",
        requires_auth=True,
        rate_limit=30
    ),
    
    # Analytics Tools
    "analytics_get_metrics": MCPTool(
        name="analytics_get_metrics",
        description="Retrieve analytics metrics for a date range",
        input_schema={
            "type": "object",
            "properties": {
                "start_date": {"type": "string", "format": "date"},
                "end_date": {"type": "string", "format": "date"},
                "metrics": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "Metrics to retrieve (pageviews, sessions, conversions, revenue)"
                }
            },
            "required": ["start_date", "end_date"]
        },
        category="analytics",
        requires_auth=True
    ),
    
    # Content Tools
    "search_knowledge_base": MCPTool(
        name="search_knowledge_base",
        description="Search the DevSkyy knowledge base using semantic search",
        input_schema={
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "Search query"},
                "top_k": {"type": "integer", "default": 5},
                "filters": {
                    "type": "object",
                    "properties": {
                        "source_type": {"type": "string"},
                        "date_after": {"type": "string", "format": "date"}
                    }
                }
            },
            "required": ["query"]
        },
        category="content"
    ),
    
    # Social Media Tools
    "instagram_schedule_post": MCPTool(
        name="instagram_schedule_post",
        description="Schedule a post to Instagram Business account",
        input_schema={
            "type": "object",
            "properties": {
                "image_url": {"type": "string", "format": "uri"},
                "caption": {"type": "string", "maxLength": 2200},
                "hashtags": {"type": "array", "items": {"type": "string"}},
                "scheduled_time": {"type": "string", "format": "date-time"}
            },
            "required": ["image_url", "caption"]
        },
        category="social_media",
        requires_auth=True,
        rate_limit=10
    ),
    
    # Email Tools
    "sendgrid_send_email": MCPTool(
        name="sendgrid_send_email",
        description="Send an email via SendGrid API",
        input_schema={
            "type": "object",
            "properties": {
                "to_email": {"type": "string", "format": "email"},
                "subject": {"type": "string"},
                "html_content": {"type": "string"},
                "template_id": {"type": "string", "description": "SendGrid template ID"}
            },
            "required": ["to_email", "subject"]
        },
        category="email",
        requires_auth=True,
        rate_limit=100
    ),
}

print(f"Total MCP tools registered: {len(MCP_TOOL_REGISTRY)}")

In [ ]:
# MCP Tool Selection Prompt (On-Demand Loading)
MCP_TOOL_SELECTION_TEMPLATE = """
You have access to the following tool categories. Based on the user's request,
determine which tools (if any) are needed.

Available Tool Categories:
{% for category, tools in tools_by_category.items() %}
- {{ category }}: {{ tools | join(', ') }}
{% endfor %}

User Request: {{ request }}

Respond with the tool names needed (comma-separated) or "none" if no tools are required.
Only request tools that are directly relevant to the task.
"""

# MCP Tool Execution Prompt
MCP_TOOL_EXECUTION_TEMPLATE = """
You are {{ agent_name }} with access to the following tools:

{% for tool in available_tools %}
<tool name="{{ tool.name }}">
Description: {{ tool.description }}
Input Schema: {{ tool.input_schema | tojson }}
{% if tool.rate_limit %}Rate Limit: {{ tool.rate_limit }} requests/minute{% endif %}
</tool>
{% endfor %}

## Instructions
1. Analyze the user's request
2. If a tool is needed, use it by responding with:
   <tool_use>
   <name>tool_name</name>
   <input>{{ "param": "value" }}</input>
   </tool_use>
3. After receiving tool results, provide a complete response
4. If no tools are needed, respond directly

User Request: {{ request }}
"""

In [ ]:
class MCPPromptBuilder:
    """Build prompts for MCP tool calling with on-demand loading."""
    
    def __init__(self, tool_registry: dict[str, MCPTool]):
        self.tool_registry = tool_registry
        self.env = Environment(loader=BaseLoader())
        self.selection_template = self.env.from_string(MCP_TOOL_SELECTION_TEMPLATE)
        self.execution_template = self.env.from_string(MCP_TOOL_EXECUTION_TEMPLATE)
    
    def get_tools_by_category(self) -> dict[str, list[str]]:
        """Group tools by category."""
        categories: dict[str, list[str]] = {}
        for tool in self.tool_registry.values():
            if tool.category not in categories:
                categories[tool.category] = []
            categories[tool.category].append(tool.name)
        return categories
    
    def build_selection_prompt(self, request: str) -> str:
        """Build prompt to determine which tools are needed (first-pass)."""
        return self.selection_template.render(
            tools_by_category=self.get_tools_by_category(),
            request=request
        )
    
    def build_execution_prompt(
        self,
        agent_name: str,
        request: str,
        tool_names: list[str]
    ) -> str:
        """Build prompt with only the required tools loaded (on-demand)."""
        available_tools = [
            self.tool_registry[name]
            for name in tool_names
            if name in self.tool_registry
        ]
        
        return self.execution_template.render(
            agent_name=agent_name,
            available_tools=[{
                "name": t.name,
                "description": t.description,
                "input_schema": t.input_schema,
                "rate_limit": t.rate_limit
            } for t in available_tools],
            request=request
        )
    
    def get_token_savings(self, loaded_tools: int) -> dict:
        """Calculate token savings from on-demand loading."""
        total_tools = len(self.tool_registry)
        # Approximate tokens per tool definition: ~150 tokens
        tokens_per_tool = 150
        
        full_load_tokens = total_tools * tokens_per_tool
        on_demand_tokens = loaded_tools * tokens_per_tool
        savings = full_load_tokens - on_demand_tokens
        
        return {
            "full_load_tokens": full_load_tokens,
            "on_demand_tokens": on_demand_tokens,
            "tokens_saved": savings,
            "reduction_percentage": f"{(savings / full_load_tokens * 100):.1f}%" if full_load_tokens > 0 else "0%"
        }


# Initialize MCP prompt builder
mcp_builder = MCPPromptBuilder(MCP_TOOL_REGISTRY)

In [ ]:
# Example: On-demand tool loading workflow
user_request = "Get our top 10 products and analyze their sales performance for last month"

# Step 1: Tool Selection (lightweight prompt to determine needed tools)
selection_prompt = mcp_builder.build_selection_prompt(user_request)
print("=" * 60)
print("STEP 1: Tool Selection Prompt")
print("=" * 60)
print(selection_prompt)

# Simulated response: "woocommerce_get_products, analytics_get_metrics"
selected_tools = ["woocommerce_get_products", "analytics_get_metrics"]

# Step 2: Execution with only selected tools loaded
execution_prompt = mcp_builder.build_execution_prompt(
    agent_name="Data Analyst Agent",
    request=user_request,
    tool_names=selected_tools
)

print("\n" + "=" * 60)
print("STEP 2: Execution Prompt (On-Demand Tools)")
print("=" * 60)
print(execution_prompt)

# Token savings calculation
savings = mcp_builder.get_token_savings(len(selected_tools))
print("\n" + "=" * 60)
print("TOKEN SAVINGS")
print("=" * 60)
print(f"Full tool registry: ~{savings['full_load_tokens']} tokens")
print(f"On-demand loaded: ~{savings['on_demand_tokens']} tokens")
print(f"Tokens saved: {savings['tokens_saved']}")
print(f"Reduction: {savings['reduction_percentage']}")

---

## 4. Dual Agent Consensus Workflow

DevSkyy uses a **consensus-based multi-agent review system** for quality assurance. This mirrors `services/consensus_orchestrator.py`.

### Workflow Architecture:
1. **Content Generator** creates initial draft
2. **Three Reviewer Agents** evaluate independently
3. **Consensus Vote**: 2+ agents flag issues = redraft
4. **Max 2 redraft iterations** with feedback
5. **Human approval** via webhook (1 hour timeout)
6. **Publish or reject** based on human decision

In [ ]:
class ReviewDecision(Enum):
    """Possible decisions from a reviewer agent."""
    APPROVE = "approve"
    REQUEST_CHANGES = "request_changes"
    REJECT = "reject"


@dataclass
class AgentReview:
    """Review from a single agent."""
    agent_id: str
    decision: ReviewDecision
    confidence: float  # 0.0 to 1.0
    feedback: str
    suggestions: list[str] = field(default_factory=list)
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())


@dataclass 
class ConsensusVote:
    """Aggregated consensus from all reviewers."""
    reviews: list[AgentReview]
    consensus_decision: ReviewDecision
    approval_count: int
    rejection_count: int
    changes_requested_count: int
    aggregated_feedback: str
    requires_redraft: bool

In [ ]:
# Dual Agent Workflow Prompt Templates

# Content Generator Prompt
CONTENT_GENERATOR_TEMPLATE = """
You are the Content Generator for The Skyy Rose Collection.

{{ brand_context }}

## Task
{{ task_description }}

## Requirements
{% for req in requirements %}
- {{ req }}
{% endfor %}

{% if previous_draft %}
## Previous Draft
{{ previous_draft }}

## Reviewer Feedback to Address
{% for feedback in reviewer_feedback %}
<feedback from="{{ feedback.agent_id }}" decision="{{ feedback.decision }}">
{{ feedback.content }}
Suggestions:
{% for suggestion in feedback.suggestions %}
- {{ suggestion }}
{% endfor %}
</feedback>
{% endfor %}

Please revise the content addressing all feedback.
{% endif %}

Output your content wrapped in <draft> tags.
"""

# Reviewer Agent Prompt
REVIEWER_AGENT_TEMPLATE = """
You are {{ reviewer_name }}, a content reviewer for The Skyy Rose Collection.

{{ brand_context }}

## Your Review Focus
{{ review_focus }}

## Content to Review
<draft>
{{ content }}
</draft>

## Review Criteria
{% for criterion in criteria %}
- {{ criterion.name }}: {{ criterion.description }} (Weight: {{ criterion.weight }})
{% endfor %}

## Instructions
1. Evaluate the content against each criterion
2. Provide your decision: APPROVE, REQUEST_CHANGES, or REJECT
3. Give a confidence score (0.0-1.0)
4. Provide specific, actionable feedback
5. List concrete suggestions for improvement

Respond in this format:
<review>
<decision>APPROVE|REQUEST_CHANGES|REJECT</decision>
<confidence>0.85</confidence>
<feedback>Your detailed feedback here</feedback>
<suggestions>
- Suggestion 1
- Suggestion 2
</suggestions>
</review>
"""

# Consensus Aggregator Prompt
CONSENSUS_AGGREGATOR_TEMPLATE = """
You are the Consensus Aggregator. Analyze the reviews from all agents and determine the final decision.

## Reviews Received
{% for review in reviews %}
<agent_review agent="{{ review.agent_id }}">
Decision: {{ review.decision }}
Confidence: {{ review.confidence }}
Feedback: {{ review.feedback }}
Suggestions: {{ review.suggestions | join('; ') }}
</agent_review>
{% endfor %}

## Consensus Rules
- 3 APPROVE votes = Final APPROVE
- 2+ REQUEST_CHANGES votes = Redraft required
- 2+ REJECT votes = Final REJECT
- Otherwise, weight by confidence scores

Provide:
1. Final consensus decision
2. Aggregated feedback for the content generator
3. Priority-ordered suggestions (deduplicated)

<consensus>
<decision>APPROVE|REQUEST_CHANGES|REJECT</decision>
<requires_redraft>true|false</requires_redraft>
<aggregated_feedback>Combined feedback</aggregated_feedback>
<priority_suggestions>
1. Most important suggestion
2. Second most important
</priority_suggestions>
</consensus>
"""

In [ ]:
class DualAgentWorkflow:
    """Orchestrate the dual-agent consensus workflow."""
    
    def __init__(self):
        self.env = Environment(loader=BaseLoader())
        self.generator_template = self.env.from_string(CONTENT_GENERATOR_TEMPLATE)
        self.reviewer_template = self.env.from_string(REVIEWER_AGENT_TEMPLATE)
        self.consensus_template = self.env.from_string(CONSENSUS_AGGREGATOR_TEMPLATE)
        
        # Define reviewer specializations
        self.reviewers = {
            "brand_voice_reviewer": {
                "name": "Brand Voice Reviewer",
                "focus": "Evaluate brand voice consistency, luxury positioning, and tone alignment.",
                "criteria": [
                    {"name": "Luxury Tone", "description": "Uses sophisticated, aspirational language", "weight": 0.3},
                    {"name": "Brand Consistency", "description": "Matches The Skyy Rose Collection voice", "weight": 0.3},
                    {"name": "Exclusivity", "description": "Conveys exclusivity and premium positioning", "weight": 0.2},
                    {"name": "Emotional Appeal", "description": "Creates desire and aspiration", "weight": 0.2}
                ]
            },
            "quality_reviewer": {
                "name": "Quality & Accuracy Reviewer",
                "focus": "Verify factual accuracy, grammar, and content quality.",
                "criteria": [
                    {"name": "Factual Accuracy", "description": "All claims are accurate and verifiable", "weight": 0.35},
                    {"name": "Grammar & Style", "description": "Impeccable grammar and professional style", "weight": 0.25},
                    {"name": "Completeness", "description": "Addresses all requirements", "weight": 0.2},
                    {"name": "Clarity", "description": "Clear, easy to understand", "weight": 0.2}
                ]
            },
            "seo_reviewer": {
                "name": "SEO & Engagement Reviewer",
                "focus": "Assess SEO optimization and engagement potential.",
                "criteria": [
                    {"name": "Keyword Integration", "description": "Natural keyword placement", "weight": 0.3},
                    {"name": "Readability", "description": "Appropriate reading level", "weight": 0.2},
                    {"name": "CTAs", "description": "Effective calls to action", "weight": 0.25},
                    {"name": "Engagement Hooks", "description": "Compelling opening and structure", "weight": 0.25}
                ]
            }
        }
    
    def build_generator_prompt(
        self,
        task_description: str,
        requirements: list[str],
        previous_draft: str | None = None,
        reviewer_feedback: list[dict] | None = None
    ) -> str:
        """Build the content generator prompt."""
        return self.generator_template.render(
            brand_context=BRAND_CONTEXT,
            task_description=task_description,
            requirements=requirements,
            previous_draft=previous_draft,
            reviewer_feedback=reviewer_feedback or []
        )
    
    def build_reviewer_prompts(self, content: str) -> dict[str, str]:
        """Build prompts for all reviewer agents."""
        prompts = {}
        for reviewer_id, config in self.reviewers.items():
            prompts[reviewer_id] = self.reviewer_template.render(
                reviewer_name=config["name"],
                brand_context=BRAND_CONTEXT,
                review_focus=config["focus"],
                content=content,
                criteria=config["criteria"]
            )
        return prompts
    
    def build_consensus_prompt(self, reviews: list[AgentReview]) -> str:
        """Build the consensus aggregation prompt."""
        return self.consensus_template.render(
            reviews=[{
                "agent_id": r.agent_id,
                "decision": r.decision.value,
                "confidence": r.confidence,
                "feedback": r.feedback,
                "suggestions": r.suggestions
            } for r in reviews]
        )
    
    def calculate_consensus(self, reviews: list[AgentReview]) -> ConsensusVote:
        """Calculate consensus from reviews."""
        approval_count = sum(1 for r in reviews if r.decision == ReviewDecision.APPROVE)
        rejection_count = sum(1 for r in reviews if r.decision == ReviewDecision.REJECT)
        changes_count = sum(1 for r in reviews if r.decision == ReviewDecision.REQUEST_CHANGES)
        
        # Determine consensus
        if approval_count == 3:
            decision = ReviewDecision.APPROVE
            requires_redraft = False
        elif rejection_count >= 2:
            decision = ReviewDecision.REJECT
            requires_redraft = False
        elif changes_count >= 2:
            decision = ReviewDecision.REQUEST_CHANGES
            requires_redraft = True
        else:
            # Weight by confidence
            weighted_scores = {
                ReviewDecision.APPROVE: sum(r.confidence for r in reviews if r.decision == ReviewDecision.APPROVE),
                ReviewDecision.REJECT: sum(r.confidence for r in reviews if r.decision == ReviewDecision.REJECT),
                ReviewDecision.REQUEST_CHANGES: sum(r.confidence for r in reviews if r.decision == ReviewDecision.REQUEST_CHANGES)
            }
            decision = max(weighted_scores, key=weighted_scores.get)
            requires_redraft = decision == ReviewDecision.REQUEST_CHANGES
        
        # Aggregate feedback
        all_feedback = [r.feedback for r in reviews]
        aggregated = "\n\n".join(f"[{r.agent_id}]: {r.feedback}" for r in reviews)
        
        return ConsensusVote(
            reviews=reviews,
            consensus_decision=decision,
            approval_count=approval_count,
            rejection_count=rejection_count,
            changes_requested_count=changes_count,
            aggregated_feedback=aggregated,
            requires_redraft=requires_redraft
        )


# Initialize workflow
workflow = DualAgentWorkflow()

In [ ]:
# Example: Full Dual Agent Workflow

# Step 1: Generate initial content
generator_prompt = workflow.build_generator_prompt(
    task_description="Write a product description for our new Midnight Velvet Evening Gown",
    requirements=[
        "200-350 words",
        "Emphasize Italian velvet and hand-sewn crystals",
        "Target high-net-worth customers",
        "Include SEO keywords: luxury evening gown, designer dress, special occasion",
        "HTML format with semantic structure"
    ]
)

print("=" * 60)
print("STEP 1: Content Generator Prompt")
print("=" * 60)
print(generator_prompt[:1000] + "...")

# Simulated draft output
simulated_draft = """
<h2>The Midnight Velvet Evening Gown</h2>
<p>Embrace the allure of midnight sophistication with our exquisite Velvet Evening Gown, 
a masterpiece crafted for those who command attention at every soirée.</p>
<p>Meticulously fashioned from the finest Italian velvet, this luxury evening gown 
cascades elegantly to the floor, creating a silhouette of timeless grace...</p>
"""

In [ ]:
# Step 2: Generate reviewer prompts
reviewer_prompts = workflow.build_reviewer_prompts(simulated_draft)

print("=" * 60)
print("STEP 2: Reviewer Prompts")
print("=" * 60)

for reviewer_id, prompt in reviewer_prompts.items():
    print(f"\n--- {reviewer_id.upper()} ---")
    print(prompt[:800] + "...\n")

In [ ]:
# Step 3: Simulate reviews and calculate consensus
simulated_reviews = [
    AgentReview(
        agent_id="brand_voice_reviewer",
        decision=ReviewDecision.APPROVE,
        confidence=0.92,
        feedback="Excellent luxury tone and brand alignment. The language evokes sophistication and exclusivity.",
        suggestions=["Consider adding more sensory language about the velvet texture"]
    ),
    AgentReview(
        agent_id="quality_reviewer",
        decision=ReviewDecision.REQUEST_CHANGES,
        confidence=0.85,
        feedback="Good overall quality, but description is too short at ~80 words. Needs expansion to meet 200-350 word requirement.",
        suggestions=[
            "Expand to meet 200-350 word requirement",
            "Add care instructions section",
            "Include sizing information"
        ]
    ),
    AgentReview(
        agent_id="seo_reviewer",
        decision=ReviewDecision.REQUEST_CHANGES,
        confidence=0.78,
        feedback="Keywords present but could be better integrated. Missing meta description and alt text suggestions.",
        suggestions=[
            "Add 'designer dress' naturally into copy",
            "Include meta description",
            "Strengthen the call-to-action"
        ]
    )
]

# Calculate consensus
consensus = workflow.calculate_consensus(simulated_reviews)

print("=" * 60)
print("STEP 3: Consensus Result")
print("=" * 60)
print(f"Decision: {consensus.consensus_decision.value.upper()}")
print(f"Approval votes: {consensus.approval_count}")
print(f"Request changes votes: {consensus.changes_requested_count}")
print(f"Reject votes: {consensus.rejection_count}")
print(f"Requires redraft: {consensus.requires_redraft}")
print(f"\nAggregated Feedback:\n{consensus.aggregated_feedback}")

In [ ]:
# Step 4: If redraft required, build new generator prompt with feedback
if consensus.requires_redraft:
    redraft_prompt = workflow.build_generator_prompt(
        task_description="Write a product description for our new Midnight Velvet Evening Gown",
        requirements=[
            "200-350 words",
            "Emphasize Italian velvet and hand-sewn crystals",
            "Target high-net-worth customers",
            "Include SEO keywords: luxury evening gown, designer dress, special occasion",
            "HTML format with semantic structure"
        ],
        previous_draft=simulated_draft,
        reviewer_feedback=[
            {
                "agent_id": r.agent_id,
                "decision": r.decision.value,
                "content": r.feedback,
                "suggestions": r.suggestions
            }
            for r in simulated_reviews
            if r.decision != ReviewDecision.APPROVE
        ]
    )
    
    print("=" * 60)
    print("STEP 4: Redraft Prompt with Feedback")
    print("=" * 60)
    print(redraft_prompt)

---

## 5. Basic Prompt Templates with Python f-strings

Simple variable substitution for straightforward prompts.

In [ ]:
def create_product_description_prompt(
    product_name: str,
    category: str,
    materials: str,
    price: float,
    current_description: str = ""
) -> str:
    """Create a luxury product description prompt."""
    return f"""Create an exquisite, ultra-luxury product description for:

Product Name: {product_name}
Current Description: {current_description or 'Premium product'}
Price: ${price:,.2f}
Category: {category}
Materials: {materials}

Requirements:
1. Use sophisticated, evocative language that speaks to affluent customers
2. Emphasize exclusivity, craftsmanship, and prestige
3. Create emotional resonance and desire
4. Highlight the investment value and timeless appeal
5. Use sensory language and vivid imagery
6. Position as a status symbol and personal expression
7. Length: 200-350 words
8. Format in elegant HTML with proper semantic structure
9. Include SEO-friendly keywords naturally
10. Match The Skyy Rose Collection's brand voice: elegant, confident, exclusive

Brand Voice: Sophisticated, aspirational, confident, exclusive, refined."""

# Example usage
prompt = create_product_description_prompt(
    product_name="Midnight Silk Evening Gown",
    category="Evening Wear",
    materials="100% Mulberry silk, hand-sewn Swarovski crystals",
    price=2495.00,
    current_description="Elegant evening dress"
)
print(prompt[:500] + "...")

---

## 6. Jinja2 Templates for Complex Prompts

Jinja2 provides powerful templating with conditionals, loops, and filters.

In [ ]:
# Initialize Jinja2 environment
jinja_env = Environment(loader=BaseLoader())

# Custom filters for DevSkyy
def format_price(value: float) -> str:
    """Format price as luxury currency."""
    return f"${value:,.2f}"

def luxury_capitalize(text: str) -> str:
    """Capitalize for luxury brand style."""
    return text.title()

jinja_env.filters['price'] = format_price
jinja_env.filters['luxury'] = luxury_capitalize

In [ ]:
# DevSkyy Agent System Prompt Template
AGENT_SYSTEM_PROMPT_TEMPLATE = """
You are {{ agent_name }}, an AI agent for The Skyy Rose Collection.

{{ brand_context }}

Your specializations:
{% for skill in skills %}
- {{ skill | luxury }}
{% endfor %}

{% if tone %}
Communication tone: {{ tone }}
{% endif %}

{% if constraints %}
Constraints:
{% for constraint in constraints %}
- {{ constraint }}
{% endfor %}
{% endif %}

Always maintain the brand's luxury positioning and sophisticated voice.
"""

system_template = jinja_env.from_string(AGENT_SYSTEM_PROMPT_TEMPLATE)

In [ ]:
# Render the template for Fashion AI Agent
fashion_agent_prompt = system_template.render(
    agent_name="Fashion Intelligence Agent",
    brand_context=BRAND_CONTEXT,
    skills=[
        "trend analysis and forecasting",
        "product styling recommendations",
        "color palette optimization",
        "competitive market analysis",
        "customer preference prediction"
    ],
    tone="Sophisticated, insightful, and fashion-forward",
    constraints=[
        "Never recommend fast fashion alternatives",
        "Always emphasize quality over quantity",
        "Maintain exclusivity in recommendations"
    ]
)

print(fashion_agent_prompt)

---

## 7. Pydantic Models for Structured Prompts

Type-safe prompt construction with validation.

In [ ]:
class ProductData(BaseModel):
    """Product data for prompt generation."""
    name: str
    category: str
    price: float = Field(gt=0)
    materials: list[str] = []
    description: str = ""
    sku: str | None = None
    
class AgentConfig(BaseModel):
    """Configuration for agent prompts."""
    agent_name: str
    model: str = "claude-sonnet-4-5-20250929"
    max_tokens: int = 4096
    temperature: float = Field(default=1.0, ge=0, le=2)
    skills: list[str] = []
    
class PromptContext(BaseModel):
    """Full context for prompt rendering."""
    agent: AgentConfig
    product: ProductData | None = None
    theme: str = "luxury_minimalist"
    brand_context: str = BRAND_CONTEXT

In [ ]:
class DevSkyPromptTemplate:
    """Reusable prompt template with Jinja2 and Pydantic."""
    
    def __init__(self, template_str: str):
        self.template = jinja_env.from_string(template_str)
    
    def render(self, context: PromptContext) -> str:
        """Render template with validated context."""
        return self.template.render(**context.model_dump())
    
    def render_dict(self, **kwargs) -> str:
        """Render template with keyword arguments."""
        return self.template.render(**kwargs)

In [ ]:
# E-commerce pricing prompt template
PRICING_TEMPLATE = """
Analyze pricing strategy for {{ product.name }}:

Current Price: {{ product.price | price }}
Category: {{ product.category }}
Materials: {{ product.materials | join(', ') }}

{% if theme in ['luxury_minimalist', 'vintage_elegance'] %}
Position as ultra-premium with price justification.
{% else %}
Consider competitive pricing with value emphasis.
{% endif %}

Provide:
1. Recommended price range
2. Price positioning rationale
3. Competitor comparison
4. Seasonal adjustment suggestions
"""

pricing_prompt = DevSkyPromptTemplate(PRICING_TEMPLATE)

# Create context with validation
context = PromptContext(
    agent=AgentConfig(
        agent_name="Pricing Optimizer",
        skills=["dynamic pricing", "market analysis"]
    ),
    product=ProductData(
        name="Artisan Leather Handbag",
        category="Accessories",
        price=1299.00,
        materials=["Italian calfskin", "Gold hardware", "Silk lining"]
    ),
    theme="luxury_minimalist"
)

print(pricing_prompt.render(context))

---

## 8. JSON Mode Prompting

Techniques to get reliable JSON output from Claude for structured data.

In [ ]:
def extract_json(response: str) -> dict:
    """Extract JSON from Claude response."""
    json_start = response.index("{")
    json_end = response.rfind("}")
    return json.loads(response[json_start : json_end + 1])

def extract_between_tags(tag: str, string: str, strip: bool = True) -> list[str]:
    """Extract content between XML-style tags."""
    ext_list = re.findall(f"<{tag}>(.+?)</{tag}>", string, re.DOTALL)
    if strip:
        ext_list = [e.strip() for e in ext_list]
    return ext_list

In [ ]:
# JSON mode with response prefilling
def get_product_analysis_json(product_name: str, category: str) -> dict:
    """Get structured product analysis using JSON prefilling."""
    
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": f"""Analyze this luxury fashion product and return JSON:
                
Product: {product_name}
Category: {category}

Return a JSON object with:
- target_demographics (list of 3 customer segments)
- price_range (object with min and max)
- style_keywords (list of 5 style descriptors)
- marketing_channels (list of recommended channels)
- competitor_brands (list of 3 similar luxury brands)"""
            },
            # Prefill to force JSON output
            {"role": "assistant", "content": "Here is the analysis:\n{"}
        ],
    ).content[0].text
    
    # Add back the prefilled "{" and parse
    return json.loads("{" + response[:response.rfind("}") + 1])

# Example usage (uncomment to run)
# analysis = get_product_analysis_json("Cashmere Wrap Coat", "Outerwear")
# pprint(analysis)

---

## 9. ReAct Pattern Prompts

Templates for iterative reasoning with Thought → Action → Observation loops.

In [ ]:
REACT_SYSTEM_PROMPT = """
You are {{ agent_name }}, an AI agent using ReAct (Reasoning + Acting) methodology.

{{ brand_context }}

For each task, follow this pattern:
1. THOUGHT: Analyze the current state and decide what to do next
2. ACTION: Choose a tool and specify the input
3. OBSERVATION: Analyze the result
4. Repeat until you have a complete answer
5. FINAL ANSWER: Provide the complete response

Available tools:
{% for tool in tools %}
- {{ tool.name }}: {{ tool.description }}
{% endfor %}

Format your responses as:
THOUGHT: [your reasoning]
ACTION: [tool_name(input)]
... wait for observation ...
THOUGHT: [analyze observation]
... continue until complete ...
FINAL ANSWER: [complete response]
"""

react_prompt = DevSkyPromptTemplate(REACT_SYSTEM_PROMPT)

In [ ]:
# Define tools for ReAct agent
tools = [
    {"name": "search_inventory", "description": "Search product inventory by criteria"},
    {"name": "get_pricing", "description": "Get current pricing for a product SKU"},
    {"name": "analyze_trends", "description": "Analyze fashion trends for a category"},
    {"name": "get_customer_segments", "description": "Get customer segment data"},
    {"name": "calculate", "description": "Perform mathematical calculations"},
]

react_system = react_prompt.render_dict(
    agent_name="Strategic Planning Agent",
    brand_context=BRAND_CONTEXT,
    tools=tools
)

print(react_system)

---

## 10. DevSkyy Prompt Template Library

A reusable library class for managing prompt templates.

In [ ]:
class DevSkyPromptLibrary:
    """Centralized prompt template library for DevSkyy agents."""
    
    def __init__(self):
        self.env = Environment(loader=BaseLoader())
        self.env.filters['price'] = lambda x: f"${x:,.2f}"
        self.env.filters['luxury'] = lambda x: x.title()
        self.templates: dict[str, Template] = {}
        self.brand_context = BRAND_CONTEXT
        self._register_default_templates()
    
    def _register_default_templates(self):
        """Register built-in templates."""
        self.register("agent_system", AGENT_SYSTEM_PROMPT_TEMPLATE)
        self.register("react_system", REACT_SYSTEM_PROMPT)
        self.register("rag_synthesis", RAG_CONTEXT_SYNTHESIS_TEMPLATE)
        self.register("mcp_execution", MCP_TOOL_EXECUTION_TEMPLATE)
        self.register("content_generator", CONTENT_GENERATOR_TEMPLATE)
        self.register("reviewer_agent", REVIEWER_AGENT_TEMPLATE)
        self.register("consensus_aggregator", CONSENSUS_AGGREGATOR_TEMPLATE)
    
    def register(self, name: str, template_str: str):
        """Register a new template."""
        self.templates[name] = self.env.from_string(template_str)
    
    def render(self, name: str, **kwargs) -> str:
        """Render a template by name."""
        if name not in self.templates:
            raise ValueError(f"Template '{name}' not found")
        
        # Always include brand context unless explicitly overridden
        if 'brand_context' not in kwargs:
            kwargs['brand_context'] = self.brand_context
        
        return self.templates[name].render(**kwargs)
    
    def list_templates(self) -> list[str]:
        """List all registered templates."""
        return list(self.templates.keys())
    
    def get_agent_prompt(self, agent_id: str) -> str:
        """Get system prompt for a registered agent."""
        if agent_id not in AGENT_REGISTRY:
            raise ValueError(f"Agent '{agent_id}' not found in registry")
        return AGENT_REGISTRY[agent_id].to_system_prompt(self.brand_context)


# Initialize library
prompt_library = DevSkyPromptLibrary()
print("Available templates:", prompt_library.list_templates())

In [ ]:
# Example: Get any agent's system prompt
print("=" * 60)
print("PRICING OPTIMIZER AGENT")
print("=" * 60)
print(prompt_library.get_agent_prompt("pricing_optimizer"))

---

## 11. Best Practices Summary

### Prompt Engineering Guidelines for DevSkyy

1. **Always include brand context** - Maintain The Skyy Rose Collection voice
2. **Use XML tags for structured output** - Easy extraction with `extract_between_tags()`
3. **Prefill for JSON mode** - Add `{"role": "assistant", "content": "{"}` for reliable JSON
4. **Type-safe with Pydantic** - Validate inputs before rendering
5. **Jinja2 for complex logic** - Conditionals, loops, and filters
6. **ReAct for multi-step tasks** - Thought → Action → Observation pattern
7. **Centralize templates** - Use `DevSkyPromptLibrary` for consistency
8. **On-demand MCP loading** - 98% token reduction by loading only needed tools
9. **Dual agent consensus** - Quality assurance through multi-agent review
10. **Iterative RAG** - Multi-hop retrieval for complex questions

---

## Conclusion

This notebook provides a comprehensive template system for DevSkyy's AI agents:

- **Complete Agent Registry** - All 20+ agents with job descriptions
- **RAG System Integration** - Iterative retrieval with context synthesis
- **MCP Tool Calling** - On-demand loading for 98% token reduction
- **Dual Agent Consensus** - Multi-agent review workflow
- **Basic templates** - Simple variable substitution
- **Jinja2 templates** - Complex logic with conditionals and loops
- **Pydantic models** - Type-safe prompt construction
- **JSON mode techniques** - Structured output
- **ReAct patterns** - Multi-step reasoning
- **Centralized library** - Consistent prompt management

All templates maintain The Skyy Rose Collection's luxury brand voice and integrate seamlessly with DevSkyy's enterprise agent architecture.